In [0]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [0]:
import time

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd 'gdrive/My Drive/Datasets/'

/content/gdrive/My Drive/Datasets


In [0]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
data = pd.read_csv('df.csv')

In [0]:
data = data[['Article_Id','Title','Content','Labels']]

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831 entries, 0 to 4830
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Article_Id  4831 non-null   int64 
 1   Title       4831 non-null   object
 2   Content     4831 non-null   object
 3   Labels      4831 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 151.1+ KB


In [0]:
data.iloc[0]['Content']

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning \nThe injured were admitted to the Government General Hospital for treatment \n\n\nSeven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital  The passengers belonged to the East and West Godavari districts of Andhra Pradesh \nThe bus  owned by private operator Yatra Genie  commenced its journey from Hyderabad at 11 30 p m  on Sunday  Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying \nThe accident happened around 2 30 a m  when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district  the daily reported  The bus hit the 

In [0]:
def bot1(data,n_clusters):
  labels = data.Labels.values
  df = data.values
  rec = []

  for i in range(n_clusters):
    # (1) indices of all the points from X that belong to cluster i
    C_i = np.where(labels == i)[0].tolist() 

    # (2) indices of the points from X to be sampled from cluster i
    sample_i = np.random.choice(C_i, 2) 
    
    for j in sample_i:
      rec.append(df[j][0])

  rec = random.sample(rec,10)
  return rec

In [0]:
def bot2(data,n_clusters,articles_read):

  #get recommendations from bot1
  rec = bot1(data,n_clusters)

  #find intersection of recommendations and articles read
  intersect = np.intersect1d(rec,articles_read)

  #repeat the process until intersection is empty
  while len(intersect) != 0:
    rec = bot1(data,n_clusters)
    intersect = np.intersect1d(rec,articles_read)
  
  return rec

In [0]:
bot1(data,5)

[2498, 2482, 3388, 2467, 625, 1769, 2253, 718, 2507, 4385]

In [0]:
def clickStream(n_users=100000):
  c_data = {}

  for i in range(n_users):
    curr_user = {}

    #here we are making an assumption that speed of user remains same in all the sessions
    speed = np.random.normal(loc=225,scale=10)
     
    n_sessions = np.random.poisson(lam=1) + 1

    for j in range(n_sessions):
      curr_session = {}

      #articles served to the user
      articles = bot1(data,5)

      #number of clicks
      n_clicks = int(np.random.normal(2.8,1) + 0.5)

      #position of clicks
      pos = []
      s = time.time()
      while np.unique(pos).shape[0] != n_clicks:
        c_pos = np.random.choice(a=[0,1,2,3,4,5,6,7,8,9],p=[0.325,0.176,0.114,0.081,0.071,0.059,0.049,0.047,0.041,0.037])
        pos.append(c_pos)
        e = time.time()

        if e - s > 20:
          break

      pos = np.unique(pos)
      n_clicks = np.unique(pos).shape[0]
      
      #time spent on an article
      time_spent = []

      for k in pos:
        #does the user leave at start or not
        choice = np.random.choice(a=[1,2],p=[0.29,0.71]) #if 1 it means user leaves within 15 seconds.
        
        if choice == 1:
          c_art_time = round(np.random.normal(loc=10,scale=2),2)
          
          time_spent.append(c_art_time)

        elif choice == 2:
          #stats show If they do stick around and scroll down the page,
          #fewer than one-third of those people will read beyond the first one-third of the article.
          choice_2 = np.random.choice(a=[1,2],p=[0.66,0.34])


          content =  data.iloc[k]['Content']
          n_words = len(word_tokenize(content))

          if choice_2 == 1:
            n_words = n_words/3

          c_art_time = round((n_words/speed) * 60,2) #multiplying by 60 because user speed is calculated in words/min
          time_spent.append(c_art_time)

      curr_session['articles'] = articles
      curr_session['click_pos'] = pos
      curr_session['time_spent'] = time_spent

      curr_user[j] = curr_session
    
    c_data[i] = curr_user
    if i % 10000 == 0:
      print(i)
  
  return c_data


In [0]:
a = clickStream(50000)

0
10000
20000
30000
40000


In [0]:
np.save('click_stream.npy', a) 

In [0]:
def click_stream_to_usable_format(click_stream_data):
  n_users = len(list(click_stream_data.keys()))
  n_items = 4831

  profile = np.empty((n_users,n_items))

  profile[:] = np.nan

  for user_id in list(click_stream_data.keys()):

    articles_served = []
    time_spent = []

    for session_id in list(click_stream_data[user_id].keys()):
      c_s_articles = click_stream_data[user_id][session_id]['articles']
    
      c_s_time_spent = np.zeros(10)

      click_pos = click_stream_data[user_id][session_id]['click_pos']
      times = click_stream_data[user_id][session_id]['time_spent']

      for i in range(len(click_pos)):
        c_s_time_spent[click_pos[i] % 10] = times[i]

      articles_served.extend(c_s_articles)
      time_spent.extend(c_s_time_spent)

    for j in range(len(articles_served)):
      profile[user_id][articles_served[j]] = time_spent[j]

  
  return profile

In [0]:
users_profile = click_stream_to_usable_format(a)

In [0]:
users_profile.shape

(50000, 4831)

In [0]:
def recommendations(data,user_id,click_stream_data):
  
  n_sessions = len(click_stream[user_id].keys())

  if n_sessions == 0:
    return bot1(data,5)
  
  elif n_sessions <= 2:
      users_profile = click_stream_to_usable_format(click_stream_data)
      #articles read by the user
      articles_read = np.argwhere(users_profile[i] > 0)
      n = articles_read.shape[0]
      articles_read = articles_read.reshape((n,))

      #sending articles_read because we don't want our system to recommend articles already read by the user.
      return bot2(data,5,articles_read)

  else: 
    users_profile = click_stream_to_usable_format(click_stream_data)
    
    n_users = len(list(click_stream_data.keys()))

    #calculating avg time spent
    avg_time = []
    for i in range(n_users):
      #articles read by the user
      articles_read = np.argwhere(users_profile[i] > 0)
      n = articles_read.shape[0]
      articles_read = articles_read.reshape((n,))

      #calculating avg time spent
      total_time_spent = 0
      for j in articles_read:
        total_time_spent += users_profile[i][j]
      
      if len(articles_read) == 0:
        avg_time.append(0)
      else:
        a_time = round(total_time_spent/len(articles_read),2)
        avg_time.append(a_time)

    #calculating user similarity
    
    sim = []
    for i in range(n_users):
      
      if i == user_id:
        sim.append(1)
      else:
        #articles read by ith user
        articles_read = np.argwhere(users_profile[i] > 0)
        n = articles_read.shape[0]
        articles_read = articles_read.reshape((n,))

        #articles read by curr user
        articles_read_c_u = np.argwhere(users_profile[user_id] > 0)
        n = articles_read_c_u.shape[0]
        articles_read_c_u = articles_read_c_u.reshape((n,))

        intersection = np.intersect1d(articles_read,articles_read_c_u)
        if len(intersection) == 0:
          sim.append(0)
        else:
          numerator = 0
          denominator = 0

          #for num
          for article in intersection:
            t_a_j = users_profile[i][article]
            u_a = avg_time[i]

            t_b_j = users_profile[user_id][article]
            u_b = avg_time[user_id]

            product = (t_a_j - u_a) * (t_b_j - u_b)
            numerator += product
          
          #for den
          term1 = 0
          term2 = 0

          u_a = avg_time[i]
          u_b = avg_time[user_id]

          for article in articles_read:
            diff = users_profile[i][article] - u_a
            diff_squared = np.square(diff)

            term1 += diff_squared
          
          for article in articles_read_c_u:
            diff = users_profile[user_id][article] - u_a
            diff_squared = np.square(diff)

            term2 += diff_squared

          term1 = np.sqrt(term1)
          term2 = np.sqrt(term2)

          denominator = term1 * term2

          sim.append(numerator/denominator)
      
    #prediction time spent

    #first lets find items that aren't visited by the user
    not_visited = np.argwhere(np.isnan(users_profile[user_id]))
    n = not_visited.shape[0]
    not_visited = not_visited.reshape((n,))

    u_i = avg_time[user_id]

    predictions = []
    for i in range(len(not_visited)):
      time = u_i

      item = not_visited[i]
      
      nan_locs = np.isnan(users_profile[:,item])
      inverted = np.logical_not(nan_locs)
      locs = np.argwhere(inverted)
      n = locs.shape[0]
      locs = locs.reshape((n,))
      
      num = 0
      den = 0

      #for num
      for j in locs:
        t_k_j = users_profile[j][item]
        u_j = avg_time[j]

        num += (sim[j] * (t_k_j - u_j)) 

        den += sim[j]

        
      time += (num/den)
      
      if np.isnan(time):
        time = 0
      predictions.append(time)
    
    predictions, not_visited = zip(*sorted(zip(predictions, not_visited),reverse=True))

    rec = []
    rec.extend(not_visited[:7])

    not_visited_left = not_visited[7:]

    n = len(not_visited_left)
    
    index = random.sample(range(0,n),3)

    for i in index:
      rec.append(not_visited_left[i])
    
    return rec

In [0]:
click_stream = a

In [0]:
click_stream[90]

{0: {'articles': [907, 1145, 129, 1485, 3425, 2765, 3324, 2250, 4806, 3373],
  'click_pos': array([0, 5]),
  'time_spent': [58.51, 49.81]},
 1: {'articles': [4516, 1917, 3616, 956, 1252, 290, 2369, 4175, 3263, 4198],
  'click_pos': array([0, 1, 2]),
  'time_spent': [58.51, 17.57, 59.83]},
 2: {'articles': [838, 567, 2804, 2240, 2570, 4620, 92, 4275, 3509, 4511],
  'click_pos': array([0, 3, 5]),
  'time_spent': [19.5, 28.55, 16.6]}}

In [0]:
rec = recommendations(data,90,click_stream)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:138: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
#Now lets see how our model performs

articles_read = np.argwhere(users_profile[90] > 0)
n = articles_read.shape[0]
articles_read = articles_read.reshape((n,))

for article in articles_read:
  print(data.iloc[article]['Title'])
  print()

After Rajinikanth  Ajith  Vijay and Suriya  Nayantara finally pairs up with Vikram

Update Samsung Galaxy S4 Mini with Android 5 1 1 Lollipop update via SlimLP custom ROM  How to install 

Vanitha-Cera Film Awards  Mammootty  Manju Warrier Win Best Actor Awards   1983  Best Film  PHOTOS 

Air India Offloads Passengers to Accommodate Rijiju on Flight  Minister Says He was  Unaware 

Media gag in Kashmir  Local newspapers asked to stop printing  cable TV goes off air

 Bajrangi Bhaijaan  Actor Salman Khan to Start Shooting for  Sultan   Superstar to Romance  Tanu Weds Manu Returns  Star Kangana Ranaut 

Micromax Yu Yureka vs Xiaomi Redmi Note 4G  Which is the Best under  10 000 Smartphone and Why 

ADB revises forecast for China s 2015 economic growth higher



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
for article in rec:
  print(data.iloc[article]['Title'])
  print()

Important India-UK talks on terrorism threat on Friday  says Cameron

Scoop  Renault India to launch Captur compact SUV  Laguna Sedan in 2017

LG G5 SE with Qualcomm Snapdragon 652 SoC tipped to be launched in select regions

RBI governor Rajan hopeful of GST Bill passage  National Herald case could spoil the game

 Su Su Sudhi Vathmeekam  first teaser featuring Jayasurya  Aju Varghese released  VIDEO 

Narendra Modi condemns killing of 14 Nepali guards in Kabul  Taliban claims responsibility

Masterpiece Box Office Collection  Yash starrer beats  Dilwale    Bajirao Mastani  to earn Rs 16 6 crore in first weekend

LIC Raises Exposure in 12 Sensex Firms

Usain Bolt set for his last Olympics in Rio  wants to run sub-19 in 200m

Vedalam  Vedhalam  box office collection  Ajith s star power works big time in its success

